In [1]:
!pip install -q timm onnxruntime onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 92.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.1/689.1 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 14.2 MB/s eta 0:00:00


In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
TRAIN_REAL   = "/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/train/real"
TRAIN_ATTACK = "/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/train/attack"
TEST_REAL    = "/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/test/real"
TEST_ATTACK  = "/kaggle/input/datasets/lightvvcx/full-dataset-frames/FULL_DATASET_FRAMES/test/attack"

In [4]:
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50
WARMUP_EPOCHS = 5


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [6]:
# ============================================================================
# DATA TRANSFORMS
# ============================================================================

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomResizedCrop(224, scale=(0.85, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    transforms.RandomRotation(15),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.15))
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [7]:
# ============================================================================
# DATASET
# ============================================================================

class AntispoofDataset(Dataset):
    def __init__(self, real_dir, attack_dir, transform=None):
        self.samples = []
        self.transform = transform
        
        for person_id in os.listdir(real_dir):
            person_path = os.path.join(real_dir, person_id)
            if os.path.isdir(person_path):
                for img_name in os.listdir(person_path):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append((os.path.join(person_path, img_name), 1.0))  # Real = 1
        
        for person_id in os.listdir(attack_dir):
            person_path = os.path.join(attack_dir, person_id)
            if os.path.isdir(person_path):
                for img_name in os.listdir(person_path):
                    if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.samples.append((os.path.join(person_path, img_name), 0.0))  # Attack = 0
        
        real_count = sum(1 for _, label in self.samples if label == 1.0)
        attack_count = len(self.samples) - real_count
        print(f"Loaded {len(self.samples)} frames - Real: {real_count} | Attack: {attack_count}")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.float32)

In [8]:
train_dataset = AntispoofDataset(TRAIN_REAL, TRAIN_ATTACK, train_transform)
test_dataset = AntispoofDataset(TEST_REAL, TEST_ATTACK, test_transform)

Loaded 2806 frames - Real: 1333 | Attack: 1473
Loaded 1302 frames - Real: 637 | Attack: 665


In [9]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [10]:
# ============================================================================
# LIGHTWEIGHT MODEL WITH TEXTURE BRANCH
# ============================================================================

class SEBlock(nn.Module):
    """Squeeze-and-Excitation block"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.squeeze(x).view(b, c)
        y = self.excitation(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [11]:
class TextureBlock(nn.Module):
    """Multi-scale texture extraction"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.branch_a = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 2, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels // 2),
            nn.ReLU(inplace=True)
        )
        self.branch_b = nn.Sequential(
            nn.Conv2d(in_channels, out_channels // 2, 5, padding=2, bias=False),
            nn.BatchNorm2d(out_channels // 2),
            nn.ReLU(inplace=True)
        )
        self.se = SEBlock(out_channels)
    
    def forward(self, x):
        a = self.branch_a(x)
        b = self.branch_b(x)
        out = torch.cat([a, b], dim=1)
        out = self.se(out)
        return out

In [12]:
class ResidualBlock(nn.Module):
    """Residual block with dilation"""
    def __init__(self, in_channels, out_channels, dilation=2, dropout=0.3):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=dilation, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(dropout)
        )
        self.se = SEBlock(out_channels)
        
        self.skip = nn.Sequential()
        if in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = self.conv(x)
        out = self.se(out)
        out = out + self.skip(x)
        return F.relu(out)


In [13]:
class AntispoofCNN(nn.Module):
    """
    Lightweight Anti-spoofing CNN with Texture Branch
    Binary classification: Single logit output
    """
    def __init__(self):
        super().__init__()
        
        # Stem
        self.stem = nn.Sequential(
            nn.Conv2d(3, 32, 5, stride=2, padding=2, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        
        # Multi-scale texture extraction
        self.texture_block = TextureBlock(32, 64)
        self.pool1 = nn.AvgPool2d(2, 2)
        
        # Main backbone
        self.block2 = ResidualBlock(64, 128, dilation=2, dropout=0.3)
        self.pool2 = nn.AvgPool2d(2, 2)
        
        self.block3 = ResidualBlock(128, 256, dilation=2, dropout=0.4)
        self.pool3 = nn.AvgPool2d(2, 2)
        
        self.block4 = ResidualBlock(256, 512, dilation=2, dropout=0.5)
        
        # Specialized texture analysis branch (from block2)
        self.texture_analyzer = nn.Sequential(
            nn.Conv2d(128, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3)
        )
        
        # Global pooling for main branch
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        
        # Classifier: Combines main features + texture features
        self.classifier = nn.Sequential(
            nn.Linear(512 + 64, 256),  # Main (512) + Texture (64)
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.4),
            nn.Linear(128, 1)  # SINGLE LOGIT OUTPUT
        )
    
    def forward(self, x):
        x = self.stem(x)
        x = self.texture_block(x)
        x = self.pool1(x)
        
        x = self.block2(x)
        texture_feat = self.texture_analyzer(x)  # Extract texture features
        x = self.pool2(x)
        
        x = self.block3(x)
        x = self.pool3(x)
        x = self.block4(x)
        
        main_feat = self.global_pool(x).flatten(1)
        
        # Combine main + texture features
        combined = torch.cat([main_feat, texture_feat], dim=1)
        
        # Single logit output
        output = self.classifier(combined)
        
        return output.squeeze()  # Remove extra dimensions

In [14]:
model = AntispoofCNN().to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"\nModel parameters: {total_params:,}")


Model parameters: 2,100,577


In [19]:
# ============================================================================
# TRAINING SETUP - BCE LOSS FOR BINARY CLASSIFICATION
# ============================================================================

# Calculate class weights for imbalance
train_real_count = sum(1 for _, label in train_dataset.samples if label == 1.0)
train_attack_count = len(train_dataset.samples) - train_real_count
pos_weight = torch.tensor([train_attack_count / train_real_count]).to(device)

print(f"\nClass balance - Real: {train_real_count}, Attack: {train_attack_count}")
print(f"Positive weight (for Real class): {pos_weight.item():.3f}")


Class balance - Real: 1333, Attack: 1473
Positive weight (for Real class): 1.105


In [20]:
# BCE Loss with positive weight
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS - WARMUP_EPOCHS)

# Mixed precision
scaler = torch.cuda.amp.GradScaler()

In [21]:
# ============================================================================
# TRAINING FUNCTIONS
# ============================================================================

def train_epoch(model, loader, criterion, optimizer, scaler):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training')
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
        preds = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (preds == labels).sum().item()
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{100.*correct/total:.2f}%'})
    
    return running_loss / len(loader), 100. * correct / total

In [22]:
def validate_with_metrics(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    real_correct = 0
    real_total = 0
    attack_correct = 0
    attack_total = 0
    
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validation'):
            images, labels = images.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            probs = torch.sigmoid(outputs)
            preds = (probs > 0.5).float()
            
            total += labels.size(0)
            correct += (preds == labels).sum().item()
            
            # Per-class accuracy
            real_mask = labels == 1.0
            attack_mask = labels == 0.0
            
            real_total += real_mask.sum().item()
            attack_total += attack_mask.sum().item()
            
            real_correct += (preds[real_mask] == labels[real_mask]).sum().item()
            attack_correct += (preds[attack_mask] == labels[attack_mask]).sum().item()
            
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    val_loss = running_loss / len(loader)
    val_acc = 100. * correct / total
    real_acc = 100. * real_correct / real_total if real_total > 0 else 0
    attack_acc = 100. * attack_correct / attack_total if attack_total > 0 else 0
    auc = roc_auc_score(all_labels, all_probs)
    
    return val_loss, val_acc, real_acc, attack_acc, auc

In [23]:
# ============================================================================
# TRAINING LOOP
# ============================================================================

best_score = float('inf')
best_epoch = 0

print("\n" + "="*60)
print("STARTING TRAINING - BINARY CLASSIFICATION (BCE LOSS)")
print("="*60)

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("-" * 60)
    
    # Warmup
    if epoch < WARMUP_EPOCHS:
        lr = LEARNING_RATE * (epoch + 1) / WARMUP_EPOCHS
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler)
    
    # Validate
    val_loss, val_acc, real_acc, attack_acc, auc = validate_with_metrics(model, test_loader, criterion)
    
    # Update scheduler
    if epoch >= WARMUP_EPOCHS:
        scheduler.step()
    
    current_lr = optimizer.param_groups[0]['lr']
    divergence = abs(real_acc - attack_acc)
    
    print(f"\nTrain Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    print(f"Real: {real_acc:.2f}% | Attack: {attack_acc:.2f}% | Divergence: {divergence:.2f}%")
    print(f"AUC: {auc:.4f}")
    print(f"Learning Rate: {current_lr:.6f}")
    
    # Save best model (lowest divergence + highest accuracy)
    score = divergence - val_acc
    
    if score < best_score:
        best_score = score
        best_epoch = epoch + 1
        torch.save(model.state_dict(), "/kaggle/working/best_optimized_cnn.pth")
        print(f"✓ Best model saved! (Divergence: {divergence:.2f}%, Val Acc: {val_acc:.2f}%, AUC: {auc:.4f})")

print("\n" + "="*60)
print(f"TRAINING COMPLETE - Best Epoch: {best_epoch}")
print("="*60)


STARTING TRAINING - BINARY CLASSIFICATION (BCE LOSS)

Epoch 1/50
------------------------------------------------------------


Validation: 100%|██████████| 41/41 [00:04<00:00,  8.26it/s]



Train Loss: 0.6638 | Train Acc: 65.11%
Val Loss: 0.5774 | Val Acc: 74.58%
Real: 88.85% | Attack: 60.90% | Divergence: 27.95%
AUC: 0.8280
Learning Rate: 0.000200
✓ Best model saved! (Divergence: 27.95%, Val Acc: 74.58%, AUC: 0.8280)

Epoch 2/50
------------------------------------------------------------


Validation: 100%|██████████| 41/41 [00:02<00:00, 19.92it/s]



Train Loss: 0.6511 | Train Acc: 66.82%
Val Loss: 0.5348 | Val Acc: 79.80%
Real: 91.52% | Attack: 68.57% | Divergence: 22.95%
AUC: 0.8564
Learning Rate: 0.000400
✓ Best model saved! (Divergence: 22.95%, Val Acc: 79.80%, AUC: 0.8564)

Epoch 3/50
------------------------------------------------------------


...


Train Loss: 0.3867 | Train Acc: 83.18%
Val Loss: 0.4273 | Val Acc: 81.34%
Real: 81.32% | Attack: 81.35% | Divergence: 0.03%
AUC: 0.9100
Learning Rate: 0.000005
✓ Best model saved! (Divergence: 0.03%, Val Acc: 81.34%, AUC: 0.9100)

Epoch 49/50
------------------------------------------------------------


Validation: 100%|██████████| 41/41 [00:02<00:00, 18.41it/s]



Train Loss: 0.3765 | Train Acc: 83.82%
Val Loss: 0.4461 | Val Acc: 81.57%
Real: 78.34% | Attack: 84.66% | Divergence: 6.33%
AUC: 0.9089
Learning Rate: 0.000001

Epoch 50/50
------------------------------------------------------------


Validation: 100%|██████████| 41/41 [00:02<00:00, 17.72it/s]


Train Loss: 0.3760 | Train Acc: 83.50%
Val Loss: 0.4427 | Val Acc: 81.34%
Real: 80.22% | Attack: 82.41% | Divergence: 2.19%
AUC: 0.9059
Learning Rate: 0.000000

TRAINING COMPLETE - Best Epoch: 48


In [24]:
# ============================================================================
# FINAL EVALUATION
# ============================================================================

print("\n\nEvaluating best model...")
model.load_state_dict(torch.load("/kaggle/working/best_optimized_cnn.pth"))
_, val_acc, real_acc, attack_acc, auc = validate_with_metrics(model, test_loader, criterion)

print(f"\n{'='*60}")
print(f"FINAL TEST RESULTS (BEST MODEL FROM EPOCH {best_epoch})")
print(f"{'='*60}")
print(f"Overall Accuracy: {val_acc:.2f}%")
print(f"Real Detection: {real_acc:.2f}%")
print(f"Attack Detection: {attack_acc:.2f}%")
print(f"Divergence: {abs(real_acc - attack_acc):.2f}%")
print(f"AUC Score: {auc:.4f}")
print(f"{'='*60}")



Evaluating best model...


Validation: 100%|██████████| 41/41 [00:02<00:00, 19.82it/s]


FINAL TEST RESULTS (BEST MODEL FROM EPOCH 48)
Overall Accuracy: 81.34%
Real Detection: 81.32%
Attack Detection: 81.35%
Divergence: 0.03%
AUC Score: 0.9100


In [25]:
# ============================================================================
# ONNX EXPORT
# ============================================================================

print("\n\nExporting to ONNX...")
dummy_input = torch.randn(1, 3, 224, 224).to(device)
onnx_path = "/kaggle/working/optimized_antispoof_cnn.onnx"

torch.onnx.export(
    model, dummy_input, onnx_path,
    export_params=True, opset_version=13, do_constant_folding=True,
    input_names=["input"], output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}
)

print(f"✓ ONNX: {onnx_path}")
print(f"✓ PyTorch: /kaggle/working/best_optimized_cnn.pth")
print(f"\n🎯 Complete! Model outputs SINGLE LOGIT for binary classification")



Exporting to ONNX...
✓ ONNX: /kaggle/working/optimized_antispoof_cnn.onnx
✓ PyTorch: /kaggle/working/best_optimized_cnn.pth

🎯 Complete! Model outputs SINGLE LOGIT for binary classification


In [26]:
# ============================================================================
# ONNX VERIFICATION
# ============================================================================

print("\n" + "="*60)
print("VERIFYING ONNX MODEL")
print("="*60)

# Install onnxruntime if needed
try:
    import onnxruntime as ort
except:
    import subprocess
    subprocess.check_call(['pip', 'install', 'onnxruntime', '--quiet'])
    import onnxruntime as ort

# Load ONNX model
ort_session = ort.InferenceSession(onnx_path)
print(f"✓ ONNX model loaded")
print(f"  Inputs: {[i.name for i in ort_session.get_inputs()]}")
print(f"  Outputs: {[o.name for o in ort_session.get_outputs()]}")

# Test with random input
print("\n1. Testing with random input...")
test_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# ONNX inference
onnx_output = ort_session.run(None, {"input": test_input})[0]

# PyTorch inference
model.eval()
with torch.no_grad():
    torch_output = model(torch.from_numpy(test_input).to(device)).cpu().numpy()

print(f"ONNX output:    {onnx_output}")
print(f"PyTorch output: {torch_output}")
print(f"Difference:     {np.abs(onnx_output - torch_output).max():.6f}")

if np.abs(onnx_output - torch_output).max() < 1e-4:
    print("✓ ONNX and PyTorch outputs MATCH (difference < 0.0001)")
else:
    print("✗ WARNING: ONNX and PyTorch outputs DIFFER significantly!")

# Test with actual test images
print("\n2. Testing with actual test images...")
num_test_samples = 5
test_samples = []

# Get some real test images
for images, labels in test_loader:
    test_samples.append((images[:num_test_samples], labels[:num_test_samples]))
    break

images, labels = test_samples[0]
images_np = images.numpy()

# ONNX predictions
onnx_preds = []
for i in range(num_test_samples):
    img = images_np[i:i+1]
    onnx_out = ort_session.run(None, {"input": img})[0][0]
    onnx_prob = 1 / (1 + np.exp(-onnx_out))
    onnx_preds.append(onnx_prob)

# PyTorch predictions
torch_preds = []
model.eval()
with torch.no_grad():
    for i in range(num_test_samples):
        img = images[i:i+1].to(device)
        torch_out = model(img).cpu().numpy()[0]
        torch_prob = 1 / (1 + np.exp(-torch_out))
        torch_preds.append(torch_prob)

# Compare predictions
print("\nSample predictions comparison:")
print(f"{'Label':<10} {'ONNX Prob':<15} {'PyTorch Prob':<15} {'Difference':<15}")
print("-" * 60)
for i in range(num_test_samples):
    label_str = "Real" if labels[i] == 1.0 else "Attack"
    diff = abs(onnx_preds[i] - torch_preds[i])
    print(f"{label_str:<10} {onnx_preds[i]:<15.4f} {torch_preds[i]:<15.4f} {diff:<15.6f}")

avg_diff = np.mean([abs(o - t) for o, t in zip(onnx_preds, torch_preds)])
print(f"\nAverage probability difference: {avg_diff:.6f}")

if avg_diff < 0.01:
    print("✓ ONNX model verified successfully!")
else:
    print("✗ WARNING: ONNX predictions differ from PyTorch!")

print("\n" + "="*60)
print("VERIFICATION COMPLETE")
print("="*60)


VERIFYING ONNX MODEL
✓ ONNX model loaded
  Inputs: ['input']
  Outputs: ['output']

1. Testing with random input...
ONNX output:    21.0582218170166
PyTorch output: 21.058225631713867
Difference:     0.000004
✓ ONNX and PyTorch outputs MATCH (difference < 0.0001)

2. Testing with actual test images...


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [27]:
# ============================================================================
# ONNX VERIFICATION (FIXED)
# ============================================================================

print("\n" + "="*60)
print("VERIFYING ONNX MODEL")
print("="*60)

# Install onnxruntime if needed
try:
    import onnxruntime as ort
except:
    import subprocess
    subprocess.check_call(['pip', 'install', 'onnxruntime', '--quiet'])
    import onnxruntime as ort

# Load ONNX model
ort_session = ort.InferenceSession(onnx_path)
print(f"✓ ONNX model loaded")
print(f"  Inputs: {[i.name for i in ort_session.get_inputs()]}")
print(f"  Outputs: {[o.name for o in ort_session.get_outputs()]}")

# Test with random input
print("\n1. Testing with random input...")
test_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# ONNX inference
onnx_output = ort_session.run(None, {"input": test_input})[0]
print(f"ONNX output shape: {onnx_output.shape}")

# PyTorch inference
model.eval()
with torch.no_grad():
    torch_output = model(torch.from_numpy(test_input).to(device)).cpu().numpy()

print(f"ONNX output:    {onnx_output}")
print(f"PyTorch output: {torch_output}")
print(f"Difference:     {np.abs(onnx_output - torch_output).max():.6f}")

if np.abs(onnx_output - torch_output).max() < 1e-4:
    print("✓ ONNX and PyTorch outputs MATCH (difference < 0.0001)")
else:
    print("✗ WARNING: ONNX and PyTorch outputs DIFFER significantly!")

# Test with actual test images
print("\n2. Testing with actual test images...")
num_test_samples = 5
test_samples = []

# Get some real test images
for images, labels in test_loader:
    test_samples.append((images[:num_test_samples], labels[:num_test_samples]))
    break

images, labels = test_samples[0]
images_np = images.numpy()

# ONNX predictions
onnx_preds = []
for i in range(num_test_samples):
    img = images_np[i:i+1]
    onnx_out = ort_session.run(None, {"input": img})[0]
    
    # Handle different output shapes
    if onnx_out.ndim == 0:  # Scalar
        onnx_logit = float(onnx_out)
    elif onnx_out.shape == (1,):  # Shape (1,)
        onnx_logit = onnx_out[0]
    else:  # Shape (1, 1) or other
        onnx_logit = onnx_out.flatten()[0]
    
    onnx_prob = 1 / (1 + np.exp(-onnx_logit))
    onnx_preds.append(onnx_prob)

# PyTorch predictions
torch_preds = []
model.eval()
with torch.no_grad():
    for i in range(num_test_samples):
        img = images[i:i+1].to(device)
        torch_out = model(img).cpu().numpy()
        
        # Handle different output shapes
        if torch_out.ndim == 0:
            torch_logit = float(torch_out)
        else:
            torch_logit = torch_out.flatten()[0]
        
        torch_prob = 1 / (1 + np.exp(-torch_logit))
        torch_preds.append(torch_prob)

# Compare predictions
print("\nSample predictions comparison:")
print(f"{'Label':<10} {'ONNX Prob':<15} {'PyTorch Prob':<15} {'Difference':<15}")
print("-" * 60)
for i in range(num_test_samples):
    label_str = "Real" if labels[i] == 1.0 else "Attack"
    diff = abs(onnx_preds[i] - torch_preds[i])
    print(f"{label_str:<10} {onnx_preds[i]:<15.4f} {torch_preds[i]:<15.4f} {diff:<15.6f}")

avg_diff = np.mean([abs(o - t) for o, t in zip(onnx_preds, torch_preds)])
print(f"\nAverage probability difference: {avg_diff:.6f}")

if avg_diff < 0.01:
    print("✓ ONNX model verified successfully!")
else:
    print("✗ WARNING: ONNX predictions differ from PyTorch!")

print("\n" + "="*60)
print("VERIFICATION COMPLETE")
print("="*60)


VERIFYING ONNX MODEL
✓ ONNX model loaded
  Inputs: ['input']
  Outputs: ['output']

1. Testing with random input...
ONNX output shape: ()
ONNX output:    21.19365882873535
PyTorch output: 21.193660736083984
Difference:     0.000002
✓ ONNX and PyTorch outputs MATCH (difference < 0.0001)

2. Testing with actual test images...

Sample predictions comparison:
Label      ONNX Prob       PyTorch Prob    Difference     
------------------------------------------------------------
Real       0.8358          0.8358          0.000000       
Real       0.8827          0.8827          0.000000       
Real       0.6714          0.6714          0.000000       
Real       0.6794          0.6794          0.000000       
Real       0.5353          0.5353          0.000002       

Average probability difference: 0.000000
✓ ONNX model verified successfully!

VERIFICATION COMPLETE
